In [ ]:
import os
import urllib

In [ ]:
import dolfinx.io
import dolfinx.mesh
import mpi4py.MPI
import numpy as np
import numpy.typing as npt
import packaging.version

In [ ]:
import viskex

Read in and plot mesh

In [ ]:
msh_filename = "data/garda.msh"
if not os.path.isfile(msh_filename):
    os.makedirs("data", exist_ok=True)
    msh_url = (
        "https://raw.githubusercontent.com/FEMlium/FEMlium/main/"
        "tutorials/01_introduction/data/garda.msh")
    with urllib.request.urlopen(msh_url) as response, open(msh_filename, "wb") as msh_file:
        msh_file.write(response.read())

In [ ]:
if packaging.version.Version(dolfinx.__version__) >= packaging.version.Version("0.10.0"):
    mesh, subdomains, boundaries, *_ = dolfinx.io.gmsh.read_from_msh(  # type: ignore[attr-defined, misc, unused-ignore]
        "data/garda.msh", comm=mpi4py.MPI.COMM_WORLD, rank=0, gdim=2)
else:
    mesh, subdomains, boundaries = dolfinx.io.gmshio.read_from_msh(  # type: ignore[attr-defined, misc, unused-ignore]
        "data/garda.msh", comm=mpi4py.MPI.COMM_WORLD, rank=0, gdim=2)
assert subdomains is not None
assert boundaries is not None

In [ ]:
viskex.dolfinx.plot_mesh(mesh)

In [ ]:
viskex.dolfinx.plot_mesh(mesh, dim=2)

In [ ]:
viskex.dolfinx.plot_mesh(mesh, dim=1)

In [ ]:
viskex.dolfinx.plot_mesh(mesh, dim=0)

Plot subdomains

In [ ]:
viskex.dolfinx.plot_mesh_tags(mesh, subdomains, "subdomains")

Plot boundaries

In [ ]:
viskex.dolfinx.plot_mesh_tags(mesh, boundaries, "boundaries")

Plot a scalar field

In [ ]:
scalar_function_space = dolfinx.fem.functionspace(mesh, ("CG", 2))

In [ ]:
centroid = np.array([631544.0, 5054515.0])

In [ ]:
def scalar_field_eval(x: npt.NDArray[np.float64]) -> npt.NDArray[np.float64]:
    """Evaluate the scalar field."""
    rho = np.sqrt((x[0] - centroid[0])**2 + (x[1] - centroid[1])**2)
    theta = np.arctan2(x[1] - centroid[1], x[0] - centroid[0])
    return rho / np.sqrt(1 - 0.5 * np.cos(theta)**2)  # type: ignore[no-any-return]

In [ ]:
scalar_field = dolfinx.fem.Function(scalar_function_space)
scalar_field.interpolate(scalar_field_eval)

In [ ]:
viskex.dolfinx.plot_scalar_field(scalar_field, "scalar")

Plot a vector field

In [ ]:
vector_function_space = dolfinx.fem.functionspace(mesh, ("CG", 2, (mesh.geometry.dim, )))

In [ ]:
def vector_field_eval(x: npt.NDArray[np.float64]) -> npt.NDArray[np.float64]:
    """Evaluate the vector field."""
    rho = np.sqrt((x[0] - centroid[0])**2 + (x[1] - centroid[1])**2)
    theta = np.arctan2(x[1] - centroid[1], x[0] - centroid[0])
    values = np.zeros((2, x.shape[1]))
    values[0] = - rho * np.sin(theta)
    values[1] = rho * np.cos(theta)
    return values

In [ ]:
vector_field = dolfinx.fem.Function(vector_function_space)
vector_field.interpolate(vector_field_eval)

In [ ]:
viskex.dolfinx.plot_vector_field(vector_field, "vector")

In [ ]:
viskex.dolfinx.plot_vector_field(vector_field, "vector", glyph_factor=0.1)